In [82]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split

In [83]:
os.getcwd()

'/Users/chance/Desktop/LH_home/ksc'

In [121]:
folder = os.listdir()

file = []
for f in folder:
    if "실거래가" in f:
        data = pd.read_csv("./" + f, encoding="cp949", skiprows=15)
        data = data[list(data.columns[:11])]
        file.append(data)

gajwa = pd.concat(file)
gajwa = gajwa.dropna(axis=0)

rate = pd.read_csv("./지역별_전월세전환율_20221112193907.csv", encoding="cp949")
rate = rate.T
rate = rate[2:]

# 도로조건을 정수로 변환
road=[]
for a in gajwa["도로조건"]:
    if a=="-":
        a = "0"
    road.append(int(a.split("m")[0]))
gajwa["도로조건"] = road


# 년월 column 생성
year=[]
month=[]
for d in gajwa["계약년월"]:
    day = pd.to_datetime(d, format="%Y%m")
    year.append(day.year)
    month.append(day.month)

gajwa["year"] = year
gajwa["month"] = month

built_day = [pd.to_datetime(d, format="%Y").year for d in gajwa["건축년도"]]
gajwa["건축년도"] = built_day


#원핫인코딩으로 전월세 구분하기
gajwa = pd.get_dummies(gajwa, columns = ["전월세구분"])

# 계약년월 기준으로 전월세전환율 입력하기
month = []
for r in rate.index:
    new = "".join(r.split("."))
    month.append(int(new))

rate["계약년월"] = month
rate.columns = ["전월세전환율", "계약년월"]
rate.head()

gajwa = pd.merge(left=gajwa, right=rate, how="inner", on="계약년월")

keep = [int("".join(k.split(","))) for k in gajwa["보증금(만원)"]]
gajwa["보증금(만원)"] = keep

gajwa

,시군구,번지,도로조건,계약면적(㎡),계약년월,계약일,보증금(만원),월세(만원),건축년도,도로명,year,month,전월세구분_월세,전월세구분_전세,전월세전환율
0,경상남도 진주시 가좌동,1***,12,15.0,202107,4,150,30,2016,가좌길48번길,2021,7,1,0,6.4
1,경상남도 진주시 가좌동,1***,12,19.0,202107,17,200,30,2019,가좌길48번길,2021,7,1,0,6.4
2,경상남도 진주시 가좌동,1***,12,19.0,202107,18,5000,10,2019,가좌길48번길,2021,7,1,0,6.4
3,경상남도 진주시 가좌동,1***,8,19.8,202107,17,200,32,2017,가좌길60번길,2021,7,1,0,6.4
4,경상남도 진주시 가좌동,1***,12,20.0,202107,21,100,25,2015,가좌길48번길,2021,7,1,0,6.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1614,경상남도 진주시 가좌동,1***,8,30.0,201910,20,6000,0,2018,가좌길44번길,2019,10,0,1,7.0
1615,경상남도 진주시 가좌동,4**,12,33.0,201910,20,1500,30,2010,개양로6번길,2019,10,1,0,7.0
1616,경상남도 진주시 가좌동,5**,12,25.0,201904,5,300,33,2007,진주대로500번길,2019,4,1,0,6.0
1617,경상남도 진주시 가좌동,4**,12,27.0,201904,16,250,30,2007,개양로6번길,2019,4,1,0,6.0


In [123]:
gajwa_data = pd.get_dummies(gajwa, columns = ["도로명"])

# 전월세전환율 기준으로 보증금+월세의 가치를 전세로 치환하여 value column으로 만들어준다.
value=[round(gajwa_data["월세(만원)"][i] * 12 / (gajwa_data["전월세전환율"][i]/100) + gajwa_data["보증금(만원)"][i]) for i in range(len(gajwa_data))]
gajwa_data["전세전환가격"] = value

gajwa_data = pd.DataFrame.drop(gajwa_data, columns=["시군구", "번지", "계약일", "보증금(만원)", "월세(만원)", "전월세전환율"], axis=1)

gajwa_data

,도로조건,계약면적(㎡),계약년월,건축년도,year,month,전월세구분_월세,전월세구분_전세,도로명_가좌길,도로명_가좌길29번길,...,도로명_진주대로,도로명_진주대로404번길,도로명_진주대로500번길,도로명_진주대로516번길,도로명_진주대로542번길,도로명_진주대로568번길,도로명_진주대로647번길,도로명_진주대로671번길,도로명_진주역로96번길,전세전환가격
0,12,15.0,202107,2016,2021,7,1,0,0,0,...,0,0,0,0,0,0,0,0,0,5775
1,12,19.0,202107,2019,2021,7,1,0,0,0,...,0,0,0,0,0,0,0,0,0,5825
2,12,19.0,202107,2019,2021,7,1,0,0,0,...,0,0,0,0,0,0,0,0,0,6875
3,8,19.8,202107,2017,2021,7,1,0,0,0,...,0,0,0,0,0,0,0,0,0,6200
4,12,20.0,202107,2015,2021,7,1,0,0,0,...,0,0,0,0,0,0,0,0,0,4788
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1614,8,30.0,201910,2018,2019,10,0,1,0,0,...,0,0,0,0,0,0,0,0,0,6000
1615,12,33.0,201910,2010,2019,10,1,0,0,0,...,0,0,0,0,0,0,0,0,0,6643
1616,12,25.0,201904,2007,2019,4,1,0,0,0,...,0,0,1,0,0,0,0,0,0,6900
1617,12,27.0,201904,2007,2019,4,1,0,0,0,...,0,0,0,0,0,0,0,0,0,6250


# 머신러닝

In [124]:
X = pd.DataFrame.drop(gajwa_data, columns=gajwa_data.columns[:-1])
y = pd.DataFrame.drop(gajwa_data, columns=gajwa_data.columns[-1:])

In [125]:
x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size=2, shuffle=True)